In [2]:
import numpy as np
import tensorflow as tf
import cv2 as cv
import matplotlib.pyplot as plt
from glob import glob
import os
from tensorflow.keras.models import load_model

In [7]:


new_model = load_model('/Users/mauryadarsh07gmail.com/Downloads/DeepLearning/LowLit_denoise/models/CBDNet_lowLit_refined.h5')


In [10]:
def check_pixel_range(image, x):
  """
  This function checks if 99.7% of pixels in an image lie within the range [0, x].

  Args:
      image: The input image as a NumPy array.
      x: The upper bound of the pixel range.

  Returns:
      True if 99.7% of pixels fall within the range, False otherwise.
  """

  # Flatten the image to a 1D array for easier processing
  flat_image = image.flatten()

  # Count the number of pixels within the range
  in_range = np.count_nonzero((flat_image >= 0) & (flat_image <= x))

  # Calculate the percentage of pixels in range
  total_pixels = flat_image.size
  percentage_in_range = (in_range / total_pixels) * 100

  # Check if 99.7% of pixels are within the range
  return percentage_in_range >= 99.7

def imgPrep(img):
   
    l=img
    
    
    r,g,b=cv.split(l)
    xr=0
    xg=0
    xb=0
    for i in range(0,255,5):
        if(check_pixel_range(r,i)):
            xr=i
            break
    for i in range(0,255,5):
        if(check_pixel_range(g,i)):
            xg=i
            break
    for i in range(0,255,5):
        if(check_pixel_range(b,i)):
            xb=i
            break
   

    d=np.array(r,dtype='float64')
    d/=xr
    d=np.ceil(d*300)
    d=np.clip(d,0,255)
    d=np.array(d,dtype='uint8')
    r=d


    d=np.array(g,dtype='float64')
    d/=xg
    d=np.ceil(d*300)
    d=np.clip(d,0,255)
    d=np.array(d,dtype='uint8')
    g=d


    d=np.array(b,dtype='float64')
    d/=xb
    d=np.ceil(d*300)
    d=np.clip(d,0,255)
    d=np.array(d,dtype='uint8')
    b=d

    fin=cv.merge((r,g,b))
   

    return fin



def processbef(x):
  xte1=cv.cvtColor(x,cv.COLOR_BGR2RGB)
  xte1=cv.resize(xte1,(512,512))
  xte=tf.image.convert_image_dtype(xte1,tf.float32)
  # plt.imshow(xte)
  xte=tf.expand_dims(xte,axis=0)
  ypd=new_model.predict(xte,verbose=0)
  ypd=ypd[0]
  ypd=np.ceil(ypd*255)
  ypd=np.clip(ypd,0,255)
  ypd=ypd.astype('uint8')

  ypd1=imgPrep(ypd)

  #   using histogram equilization to get some normal light component
  
  hsv=cv.cvtColor(xte1,cv.COLOR_RGB2HSV)
  h,s,v=cv.split(hsv)
  v=cv.equalizeHist(v)
  hsv=cv.merge((h,s,v))
  rgb=cv.cvtColor(hsv,cv.COLOR_HSV2RGB)

  ypd1=cv.addWeighted(ypd1,0.7,rgb,0.3,5)
  ypd1=cv.GaussianBlur(ypd1,(3,3),1)
  
  return ypd1



In [16]:
# Define paths for data directories
test_low_dir = './test/low/'
test_predicted_dir = './test/predicted/'

# Ensure output directory exists
os.makedirs(test_predicted_dir, exist_ok=True)  # Create predicted directory if it doesn't exist

model = new_model 

# Load test images from the low-light directory
test_low_images = []
for filename in os.listdir(test_low_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust for image formats
        image_path = os.path.join(test_low_dir, filename)
        image = cv.imread(image_path)  
        test_low_images.append(image)


for i, test_image in enumerate(test_low_images):
    predicted_image = processbef(test_image)  
    predicted_image=cv.cvtColor(predicted_image,cv.COLOR_RGB2BGR) #opencv reads and writes image in BGR
    # Save the predicted image
    predicted_image_path = os.path.join(test_predicted_dir, f'predicted_{i+1}.jpg')  # Adjust filename format
    cv.imwrite( predicted_image_path,predicted_image) 
